imports

In [1]:
import numpy as np
from emnist import extract_training_samples, extract_test_samples
from scipy.linalg import solve_triangular

ModuleNotFoundError: No module named 'numpy'

Load Data

In [ ]:
# load data
train_images, train_labels = extract_training_samples('letters')
test_images, test_labels = extract_test_samples('letters')

# (A=0, B=1, ..., Z=25)
train_labels = train_labels - 1
test_labels = test_labels - 1

# parameters
n_classes = 26 #number of classes(A-Z)
n_train_per_class = 200
n_test_per_class = 20

part 1: QR using householder

In [ ]:
def householder_qr(A):
    m, n = A.shape
    R = A.copy().astype(np.float64)
    Q = np.eye(m, dtype=np.float64)
    
    for k in range(min(m, n)):
        x = R[k:, k]
        if np.linalg.norm(x)==0:
            continue
        
        # claculate v
        e = np.zeros_like(x)
        e[0] = np.linalg.norm(x)
        v = x - e if x[0] >= 0 else x + e
        v = v / np.linalg.norm(v)
        
        # householder matrix: H = I - 2vv^T
        H = np.eye(m, dtype=np.float64)
        H[k:, k:] -= 2 * np.outer(v, v)
        
        R = H @ R
        Q = Q @ H  # Q = H1 * H2 * ... * Hk
        
    return Q, R

In [ ]:
# save QR for each class
class_matrices = {}
class_Q = {}
class_R = {}

for cls in range(n_classes):
    # 200 train for each class
    idx = np.where(train_labels == cls)[0][:n_train_per_class]
    A = train_images[idx].reshape(n_train_per_class, 28*28).T.astype(np.float64)
    
    # QR using householder
    Q, R = householder_qr(A)
    class_Q[cls] = Q
    class_R[cls] = R

In [ ]:
correct = 0
total = 0

for cls in range(n_classes):
    # 20 test for each class
    idx = np.where(test_labels == cls)[0][:n_test_per_class]
    test_data = test_images[idx].astype(np.float64)
    
    for b in test_data:
        b = b.reshape(28*28,-1)
        min_error = float('inf')
        predicted_cls = -1
        
        # least square error
        for c in range(n_classes):
            Q = class_Q[c]
            R = class_R[c]
            m,n = R.shape
            R1 = R[:n, :n]
            Q_b = Q.T@b
            Q_b = Q_b[:n]
            
            x = solve_triangular(R1, Q_b)
            reconstructed = Q @ (R @ x)
            error = np.linalg.norm(b - reconstructed)
            
            if error < min_error:
                min_error = error
                predicted_cls = c
        
        if predicted_cls == cls:
            correct += 1
        total += 1

accuracy_section1 = correct / total
print(f"accuracy 1: {accuracy_section1:.2%}")

accuracy 1: 70.58%


part 2: QR using givens

In [ ]:
def givens_rotation(a, b):
    if b == 0.0:
        c = 1.0
        s = 0.0
    else:
        if abs(b) >= abs(a):
            t = -a / b
            s_denom = np.sqrt(1.0 + t**2)
            s = 1.0 / s_denom
            c = s * t
        else:
            t = -b / a
            c_denom = np.sqrt(1.0 + t**2)
            c = 1.0 / c_denom
            s = c * t

    return c, s

def update_qr(Q, R, new_col):
    m, n = R.shape
    q = Q @ new_col
    R_augmented = np.hstack((R, q.reshape(-1, 1)))
    
    for i in range(m-1, n, -1):
        if R_augmented[i, n] == 0:
            continue
        c, s = givens_rotation(R_augmented[i-1, n], R_augmented[i, n])
        
        G = np.array([[c, -s], [s, c]])
        
        R_augmented[i - 1:i + 1, :] = G @ R_augmented[i - 1:i + 1, :]
        Q[:, i - 1:i + 1] = Q[:, i - 1:i + 1] @ G.T
        
    return Q, R_augmented

In [ ]:
n_new_samples = 20

for cls in range(n_classes):
    # 20 more train for each class
    idx = np.where(train_labels == cls)[0][n_train_per_class:n_train_per_class + n_new_samples]
    B = train_images[idx].reshape(n_new_samples, 28*28).astype(np.float64)
    
    # update
    for new_col in B:
        Q_new, R_new = update_qr(class_Q[cls], class_R[cls], new_col.reshape(28*28, -1))
        # print(Q_new.shape, R_new.shape)
        class_Q[cls] = Q_new
        class_R[cls] = R_new

In [ ]:
correct = 0
total = 0

for cls in range(n_classes):
    # 20 test for each class
    idx = np.where(test_labels == cls)[0][:n_test_per_class]
    test_data = test_images[idx].astype(np.float64)
    
    for b in test_data:
        b = b.reshape(28*28,-1)
        min_error = float('inf')
        predicted_cls = -1
        
        # least square error
        for c in range(n_classes):
            Q = class_Q[c]
            R = class_R[c]
            m,n = R.shape
            R1 = R[:n, :n]
            Q_b = Q.T@b
            Q_b = Q_b[:n]
            
            x = solve_triangular(R1, Q_b)
            reconstructed = Q @ (R @ x)
            error = np.linalg.norm(b - reconstructed)
            
            if error < min_error:
                min_error = error
                predicted_cls = c
        
        if predicted_cls == cls:
            correct += 1
        total += 1

accuracy_section2 = correct / total
print(f"accuracy 2: {accuracy_section2:.2%}")

accuracy 2: 70.19%


In [ ]:
accuracy_section1, accuracy_section2

(0.7057692307692308, 0.7019230769230769)